# OpenHPCの起動

---

OpenHPC環境を構成するVCノード、VCディスクを作成します。

## 構成

このNotebookが構築する環境の構成を図に示します。

![構成](images/ohpc-000.png)

## 準備

このNotebookを実行するための準備作業を行います。

### UnitGroup名の指定

構築環境の UnitGroup名を指定します。

> 「010-パラメータ設定.ipynb」と同じUnitGroup名(`ugroup_name`)を指定することで、既に入力したパラメータを引き継ぐことができます。

VCノードを作成時に指定した値を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'OpenHPC'

ugroup_name = 

### VCCアクセストークンの入力

VCノードを起動するにはVC Controller(VCC)にアクセスして、操作を行う必要があります。VCCにアクセスするために必要となるアクセストークンをここで入力します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2021/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

> `unfreeze`するにはNotebookのツールバーにある`unfreeze below in section`ボタンなどを利用してください。

### group_varsの読み込み

次のセルを実行すると「010-パラメータの設定.ipynb」で指定したパラメータを読み込みます。読み込むパラメータの値は、UnitGroup名に指定した 値に対応するものになります。UnitGroup名の指定が誤っていると意図したパラメータが読み込めないので注意してください。

In [ ]:
%run scripts/group.py

gvars = load_group_vars(ugroup_name)

## VCディスクの作成

マスターノードのNFSサーバが公開するファイルを配置するためのディスクを作成します。

> 「010-パラメータ設定.ipynb」でVCディスクを作成しない設定にした場合は、この節を実行してもVCディスクは作成されません。

![VCディスク](images/ohpc-007.png)

### VCディスクのUnitGroup作成

VCディスクを管理するためのUnitGroupを作成します。

UnitGroupを作成する前に、現在のUnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

VCディスクのUnitGroupを作成します。

In [ ]:
if 'nfs_disk_size' in gvars:
    ug_disk = vcp.create_ugroup(f'{ugroup_name}_disk', ugroup_type='storage')

UnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

### VCディスクの作成

VCディスクを作成します。

In [ ]:
if 'nfs_disk_size' in gvars:
    provider = gvars['vc_provider']
    spec_disk = vcp.get_spec(f'{provider}_disk', 'small')
    if gvars['vc_provider'] == 'azure':
        spec_disk.disk_size_gb = gvars['nfs_disk_size']
    if gvars['vc_provider'] == 'oracle':
        spec_disk.size_in_gbs = gvars['nfs_disk_size']
    else:
        spec_disk.size = gvars['nfs_disk_size']
    unit_disk = ug_disk.create_unit('nfs', spec_disk)

UnitGroupに作成したVCディスクの一覧を表示してみます。

In [ ]:
from IPython.display import display
if 'unit_disk' in vars() and unit_disk:
    display(unit_disk.df_nodes())

## VCノードの起動

マスターノード、計算ノードとして利用するVCノードを起動します。

### VCノードのUnitGroup作成

VCノードを管理するためのUnitGroupを作成します。

UnitGroupを作成する前に、現在のUnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

VCノードのUnitGroupを作成します。

In [ ]:
ug = vcp.create_ugroup(ugroup_name)

UnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

### マスターノード

マスターノードとして利用するVCノードを起動します。

![マスターノード](images/ohpc-008.png)

#### spec を指定する

「010-パラメータ設定.ipynb」で指定したパラメータを`spec` に設定します。

`spec`に設定するパラメータを以下に示します。

* `image`: Baseコンテナイメージ
  - Baseコンテナイメージを設定します
* `params_v`: ボリューム設定
  - Baseコンテナのボリュームを設定します
  - OpenHPCコンテナではホスト側の `/sys/fs/cgroup`, `/lib/modules` をコンテナから見えるように設定する必要があります
* `ip_addresses`: IPアドレス
  - VCノードに割り当てるプライベートIPアドレスを設定します
* `disks`: VCディスクのリスト
  - VCノードにアタッチするVCディスクのリストを設定します
* `volume_size`: ルートボリュームサイズ
  - VCノードのルートボリュームサイズを設定します
* `instance_type`: インスタンスタイプ
  - VCノードのインスタンスタイプ
* `set_ssh_publickey()`: SSHの公開鍵
  - VCノードに登録するSSHの公開鍵

In [ ]:
spec_master = vcp.get_spec(gvars['vc_provider'], gvars['master_flavor'])
spec_master.image = 'harbor.vcloud.nii.ac.jp/vcp/openhpc:master-2.6'
spec_master.params_v = [
    '/sys/fs/cgroup:/sys/fs/cgroup:ro',
    '/lib/modules:/lib/modules:ro',
]
spec_master.ip_addresses = [gvars['master_ipaddress']]
if 'ug_disk' in vars() and ug_disk:
    spec_master.disks = ug_disk.find_nodes()
else:
    spec_master.params_v.append('/srv/OpenHPC:/exports')
if gvars['vc_provider'] == 'aws':
    spec_master.volume_size = gvars['master_root_size']
    if 'master_instance_type' in gvars:
        spec_master.instance_type = gvars['master_instance_type']
elif gvars['vc_provider'] == 'azure':
    spec_master.disk_size_gb = gvars['master_root_size']
    if 'master_instance_type' in gvars:
        spec_master.vm_size = gvars['master_instance_type']
elif gvars['vc_provider'] == 'oracle':
    spec_master.boot_volume_size_in_gbs = gvars['master_root_size']
    if 'master_instance_type' in gvars:
        spec_master.shape = gvars['master_instance_type']
spec_master.set_ssh_pubkey(gvars['ssh_public_key_path'])

ここまで `spec` に設定した内容を表示してみます。

In [ ]:
print(spec_master)

ベースコンテナの環境変数に以下のパラメータを設定します。

* `SLURM_NODE_PARAMS`
    - slurm.conf に追加する計算ノードのパラメータ
* `MUNGE_KEY`
    - VCノードの `munge.key` の内容
* `NFS_DEV`
    - NFS用ディスクのデバイス名
    
さらに各ノードに対応するエントリを `/etc/hosts` に追加するためのパラメータを設定します。

In [ ]:
%run scripts/utils.py

spec_master.params_e.extend([
    f'SLURM_NODE_PARAMS={spec_env_slurm_conf(gvars)}',
    f'MUNGE_KEY={spec_env_munge_key(gvars, vcp, vcc_access_token)}',
])
if 'nfs_device' in gvars:
    spec_master.params_e.append(f'NFS_DEV={gvars["nfs_device"]}')
spec_master.hostname = gvars["master_hostname"]
spec_master.add_host = spec_add_host_list(gvars)

`spec` に設定した内容を表示してみます。

> 表示内容には、**秘匿情報**となる `munge.key` の内容を含んでいます。扱いには**注意してください**。

In [ ]:
print(spec_master)

#### VCノードの起動

マスターノードとして利用するVCノードを起動します。

> AWSで起動するにはおよそ５分程度かかります。

In [ ]:
unit_master = ug.create_unit('master', spec_master)

VCノードの状態が `RUNNING` になっていることを確認します。

> VCノードの起動に失敗して`RUNNING`以外の状態になっている場合は次のセルを実行するとエラーになります。

In [ ]:
unit_master = ug.get_unit('master')
if any([node.state != 'RUNNING' for node in unit_master.find_nodes()]):
    raise RuntimeError('ERROR: not running')

起動したVCノードの一覧を表示します。

> 上のセルがエラーになった場合も、次のセルを実行してノードの状態を確認してください。`node_state` が`BOOTING`だった場合は、単にノード起動に時間がかかっているだけである可能性があります。しばらく待ってから、ノードの状態を再確認してみてください。また、`node_state`が`ERROR`となっている場合はノードの起動に失敗しています。`ug.cleanup()` を実行して VCノードを削除してください。その後、エラー原因を解消してから「[VCノードのUnitGroup作成](#VCノードのUnitGroup作成)」以下を `unfreeze` して再実行してください。

In [ ]:
unit_master.df_nodes()

#### NFS serverの起動確認

NFS serverのサービスが実行されていることを確認します。

In [ ]:
addr = gvars['master_ipaddress']
ssh_opts = f'-o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no -i {gvars["ssh_private_key_path"]}'
!ssh-keygen -R {addr}
!ssh {ssh_opts} vcp@{addr} systemctl status nfs-server

エラーになった場合は、起動したVCノードにて `journalctl` などを実行してエラーの原因を確認してください。原因となった事象を解消した後に次のセルのコメントを外してから実行し NFSサーバのサービスを再起動してください。

> よくある事例として、起動に時間がかかりタイムアウトすることでNFSサーバが起動しないことがあります。この場合は、単に次のセルのコメントを外してサービスの再起動を行うことで問題が解消します。

In [ ]:
# !ssh {ssh_opts} vcp@{addr} sudo systemctl restart nfs-server

サービスを再起動した場合は次のセルのコメントを外して、NFSサーバのサービスが起動したことを確認してください。

In [ ]:
# !ssh {ssh_opts} vcp@{addr} systemctl status nfs-server

NFSのエクスポート設定が１つ以上あることを確認します。次のセルを実行し、エラーとならないことを確認してください。

In [ ]:
# エクスポートの設定状況を表示する
!ssh {ssh_opts} vcp@{addr} "sudo /sbin/exportfs -v"
# エクスポートの設定項目が１つ以上あることをチェックする
!test $(ssh {ssh_opts} vcp@{addr} "sudo /sbin/exportfs -v" | wc -l) -gt 0 

上のセルがエラーとなる場合は、パラメータ設定にて指定した `nfs_device` の値が正しくないことが考えられます。次のセルのコメントを外してパラメータに設定した値と、起動したVCノードにアタッチされているブロックデバイス名が一致しているかを確認してください。正しく設定されている場合は、以下の表示例のように１行目と最後の行に表示されるブロックデバイス名（この例では`xvdf`） に同じ値が表示されます。

```
nfsdevice=/dev/xvdf
NAME    MAJ:MIN RM SIZE RO TYPE MOUNTPOINT
xvda    202:0    0  60G  0 disk 
└─xvda1 202:1    0  60G  0 part /etc/hosts
xvdf    202:80   0  64G  0 disk /exports
```

異なる値が表示されている場合は設定するパラメータを修正する必要があります。
「920-OpenHPC環境の削除.ipynb」のNotebookで起動したVCノード、VCディスクを削除してから、再度「010-パラメータ設定.ipynb」のNotebookで、正しいブロックデバイス名を設定しなおしてください。

In [ ]:
# 設定されているパラメータ nfs_device の値を確認する
# print(f'nfsdevice={gvars["nfs_device"]}')

# VCノードのブロックデバイスを確認する
# !ssh {ssh_opts} vcp@{addr} lsblk 2> /dev/null

### 計算ノード

マスターノードとして利用するVCノードを起動します。

![計算ノード](images/ohpc-009.png)

#### spec を指定する

「010-パラメータ設定.ipynb」で指定したパラメータを`spec` に設定します。

`spec`に設定するパラメータを以下に示します。

* `image`: Baseコンテナイメージ
  - Baseコンテナイメージを設定します
* `params_v`: ボリューム設定
  - Baseコンテナのボリュームを設定します
  - OpenHPCコンテナではホスト側の `/sys/fs/cgroup`, `/lib/modules` をコンテナから見えるように設定する必要があります
* `ip_addresses`: IPアドレス
  - VCノードに割り当てるプライベートIPアドレスを設定します
* `volume_size`: ルートボリュームサイズ
  - VCノードのルートボリュームサイズを設定します
* `instance_type`: インスタンスタイプ
  - VCノードのインスタンスタイプ
* `set_ssh_publickey()`: SSHの公開鍵
  - VCノードに登録するSSHの公開鍵

In [ ]:
spec_compute = vcp.get_spec(gvars['vc_provider'], gvars['compute_flavor'])
spec_compute.image = 'harbor.vcloud.nii.ac.jp/vcp/openhpc:compute-2.6'
if gvars['compute_use_gpu']:
    spec_compute.gpus = 'all'
    spec_compute.image = 'harbor.vcloud.nii.ac.jp/vcp/openhpc:compute-gpu-2.6'
    if gvars['vc_provider'] != 'onpremises':
        spec_compute.cloud_image = 'niivcp-22.04.0-x86_64-ubuntu20.04-gpu-a-r3'
spec_compute.params_v = [
    '/sys/fs/cgroup:/sys/fs/cgroup:ro',
    '/lib/modules:/lib/modules:ro',
    '/opt/var/lib/docker:/var/lib/docker',
]
spec_compute.ip_addresses = [x for x in gvars['compute_etc_hosts'].keys()]
if 'compute_root_size' in gvars:
    if gvars['vc_provider'] == 'aws':
        spec_compute.volume_size = gvars['compute_root_size']
    elif gvars['vc_provider'] == 'azure':
        spec_compute.disk_size_gb = gvars['compute_root_size']
    elif gvars['vc_provider'] == 'oracle':
        spec_compute.boot_volume_size_in_gbs = gvars['compute_root_size']
if 'compute_instance_type' in gvars:
    if gvars['vc_provider'] == 'aws':
        spec_compute.instance_type = gvars['compute_instance_type']
    elif gvars['vc_provider'] == 'azure':
        spec_compute.vm_size = gvars['compute_instance_type']
    elif gvars['vc_provider'] == 'oracle':
        spec_compute.shape = gvars['compute_instance_type']
spec_compute.set_ssh_pubkey(gvars['ssh_public_key_path'])

ここまで `spec` に設定した内容を表示してみます。

In [ ]:
print(spec_compute)

ベースコンテナの環境変数に以下のパラメータを設定します。

* `MASTER_HOSTNAME`
    - マスターノードのホスト名
* `SLURM_NODE_PARAMS`
    - slurm.conf に追加する計算ノードのパラメータ
* `MUNGE_KEY`
    - VCノードの `munge.key` の内容
    
さらに各ノードに対応するエントリを `/etc/hosts` に追加するためのパラメータを設定します。

In [ ]:
%run scripts/utils.py

spec_compute.params_e.extend([
    f'MASTER_HOSTNAME={gvars["master_hostname"]}',
    f'SLURM_NODE_PARAMS={spec_env_slurm_conf(gvars)}',
    f'MUNGE_KEY={spec_env_munge_key(gvars, vcp, vcc_access_token)}',
])
spec_compute.add_host = spec_add_host_list(gvars)

`spec` に設定した内容を表示してみます。

> 表示内容には、**秘匿情報**となる `munge.key` の内容を含んでいます。扱いには**注意してください**。

In [ ]:
print(spec_compute)

#### VCノードの起動

計算ノードとして利用するVCノードを起動します。

> AWSで起動するにはおよそ３分程度かかります。

In [ ]:
unit_compute = ug.create_unit('compute', spec_compute)

VCノードの状態が `RUNNING` になっていることを確認します。

> VCノードの起動に失敗して`RUNNING`以外の状態になっている場合は次のセルを実行するとエラーになります。エラーになった場合は、`ug.cleanup()` を実行して VCノードを削除してください。その後、「[VCノードのUnitGroup作成](#VCノードのUnitGroup作成)」以下を `unfreeze` してから再実行してください。

In [ ]:
unit_compute = ug.get_unit('compute')
if any([node.state != 'RUNNING' for node in unit_compute.find_nodes()]):
    raise RuntimeError('ERROR: not running')

起動したVCノードの一覧を表示します。

In [ ]:
unit_compute.df_nodes()

## Ansibleの設定

VCノードをAnsibleで操作するための設定を行います。

まず、VCノードにSSHでログインできるようにするために `~/.ssh/known_hosts` の更新を行います。

> 何度かVCノードの起動を行うと、異なるホストが同じIPアドレスで起動するためにSSHのホストキーのチェックでエラーになる事があります。このような状況に対応するために、起動したVCノードのIPアドレスに対応するエントリを`known_hosts`ファイルから削除します。その後、`ssh-keyscan`コマンドを利用して起動したVCノードのホストキーを取得して `known_hosts`ファイルの内容を更新します。

In [ ]:
from time import sleep

def check_update_known_hosts(ipaddr):
    # VCノード起動直後だと sshd サービスが開始されておらずに known_hosts が更新されない場合がある
    # ssh-keyscan が値を取得できるまで何度かリトライする
    for x in range(10):
        out = ! echo $(ssh-keyscan {ipaddr} 2> /dev/null | wc -l)
        update_lines = int(out[0])
        if update_lines > 0:
            break
        sleep(1)
    else:
        raise RuntimeError("ERROR: timeout!")    

!mkdir -p -m 0700 ~/.ssh
!touch ~/.ssh/known_hosts
for addr in ug.find_ip_addresses():
    !ssh-keygen -R {addr}
    check_update_known_hosts(addr)
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

起動したVCノードに対応するエントリを Ansible のインベントリに登録します。

> Ansibleで操作を行うためには、操作対象のホスト(IPアドレス)をインベントリに登録する必要があります。

In [ ]:
import yaml

inventory = {
    'all': {
        'children': {
            ug.name: {
                'children': dict([
                    (
                        f'{ug.name}_{unit.name}',
                        {
                            'hosts': dict([
                                (ip, {})
                                for ip in unit.find_ip_addresses()
                            ])
                        }
                    )
                    for unit in ug.find_units()]),
                'vars': {
                    'ansible_user': 'vcp',
                    'ansible_ssh_private_key_file': gvars['ssh_private_key_path'],
                }
            }
        }
    }
}

with open('inventory.yml', 'w') as f:
    yaml.safe_dump(inventory, f, default_flow_style=False)
    
!cat inventory.yml

先程VCノードを登録したファイルをインベントリとして指定するためのAnsibleのコンフィギュレーションファイルを作成します。

> カレントディレクトリにコンフィギュレーションファイル(`ansible.cfg`)を作成すると、Ansibleを実行する際にその設定が適用されます。

In [ ]:
from pathlib import Path

inventory_path = Path('./inventory.yml').resolve()
ansible_cfg = Path('./ansible.cfg').resolve()
with ansible_cfg.open(mode='w') as f:
    f.write(f'''
[defaults]
inventory = {inventory_path}
remote_tmp = /var/tmp
timeout = 30
''')
ansible_cfg.parent.chmod(0o755)

!cat ./ansible.cfg

VCノードに対して Ansible で接続できることを確認します。

In [ ]:
!ansible {ugroup_name} -m ping

正常に接続できると以下のように表示されます。

```
XXX.XXX.XXX.XXX | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
```

VCノードに対して設定ファイルの変更やパッケージの追加を行う場合にVCノードの管理者権限が必要になる場合があります。Ansibleで管理者権限によるコマンド実行が可能かどうかを確認します。

In [ ]:
!ansible {ugroup_name} -b -a 'whoami'

## Slurmの状態を確認する

起動したVCノードで実行されているSlurmの状態を確認します。

### ノードの状態を確認する

Slurmクラスタのノードの状態を確認します。

In [ ]:
!ansible {ugroup_name}_master -a sinfo

`STATE`が`idle`と表示されていればノードが利用可能な状態になっています。

もし、`STATE`が `down` となっている場合は次のセルのコメント(先頭の `#`)を外して実行してください。

In [ ]:
# for x in gvars['compute_etc_hosts'].values():
#     !ansible {ugroup_name}_master -b -a 'scontrol update nodename={x} state=resume' || true

もう一度、ノードの状態を確認します。

In [ ]:
!ansible {ugroup_name}_master -a sinfo

### ジョブを実行する

ジョブを実行できることを確認します。

`srun`コマンドで、`hostname` コマンドを計算ノードで実行させてみます。

計算ノードが２ノードの場合、以下のような実行結果が表示されます（ホストの表示順序は入れ替わることがあります）。

```
0: c1
1: c2
```

In [ ]:
!ansible {ugroup_name}_master -a 'srun -l -N {gvars["compute_nodes"]} hostname'

次に、サンプルの `hello.c` をコンパイルして `sbatch` で実行してみます。

まず、ソースファイルをコンパイルして、実行ファイルを作成します。

In [ ]:
!ansible {ugroup_name}_master -a 'mpicc -O3 -o hello /opt/ohpc/pub/examples/mpi/hello.c'
!ansible {ugroup_name}_master -a 'ls -l hello'

ジョブスクリプトを作成します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

with TemporaryDirectory() as work_dir:
    job_file = Path(work_dir) / 'hello.job'
    with job_file.open(mode='w') as f:
        f.write(f'''#!/bin/bash

#SBATCH -J hello               # Job name
#SBATCH -o hello.%j.out         # Name of stdout output file (%j expands to jobId)
#SBATCH -N {gvars["compute_nodes"]}                  # Total number of nodes requested
#SBATCH -n {gvars["compute_nodes"]}                  # Total number of mpi tasks requested
#SBATCH -t 00:01:00           # Run time (hh:mm:ss)

# Launch MPI-based executable
prun ./hello
''')
    !cat {job_file}
    !ansible {ugroup_name}_master -m copy -a 'src={str(job_file)} dest=.'

ジョブを投入します。

In [ ]:
!ansible {ugroup_name}_master -a 'sbatch hello.job'

ジョブの出力結果を表示してみます。ジョブの実行が成功していると以下のような出力結果が得られます。

```
[prun] Master compute host = c1
[prun] Resource manager = slurm
[prun] Launch cmd = mpirun ./hello (family=openmpi4)

 Hello, world (2 procs total)
    --> Process #   0 of   2 is alive. -> c1
    --> Process #   1 of   2 is alive. -> c2
```

In [ ]:
!ansible {ugroup_name}_master -m shell -a 'cat hello.*.out'